URL = https://www.dba.dk/biler/biler/

1. Hvor mange brugte biler er der at vælge i mellem

In [8]:
import bs4
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

base_url = "https://www.dba.dk/biler/biler/"
browser = webdriver.Firefox()
browser.get(base_url)
browser.implicitly_wait(1)


In [20]:
page_source = browser.page_source

soup = bs4.BeautifulSoup(page_source, 'html.parser')
bulletin_type_cell = soup.find_all('div', {'class': 'navigator radioNavigator modulePanel'})[0]
data_cells = bulletin_type_cell.find_all('small')
cars_for_sale = data_cells[0].text[1:-1]
cars_of_interest = data_cells[1].text[1:-1]
total_amount_of_cars = int(cars_for_sale) + int(cars_of_interest)
print("Amount of cars for sale:", cars_for_sale)
print("Totalt amount of car bulletin on DBA:", total_amount_of_cars)

Amount of cars for sale: 31324
Totalt amount of car bulletin on DBA: 31449


2. Udskriv alle biler af mærket Ford

In [68]:
checkbox_li = browser.find_element_by_class_name('no-ford')
a_tag_ford = checkbox_li.find_element_by_xpath('//a[@href="/biler/biler/maerke-ford/"]')
a_tag_ford.click()
#Wait for page to load
sleep(2)

amount_of_cars = 0
index = 1
while True:
    bulletin = browser.find_elements_by_class_name('dbaListing')
    for element in bulletin:
        description = element.find_elements_by_class_name('listingLink')[1].text
        model_year = element.find_element_by_xpath('.//td[@title="Modelår"]').text
        price = element.find_element_by_xpath('.//td[@title="Pris"]').text
        # Commented out to spare github from a wall of text:
        # print("Car description:\n", description)
        print("Model year: ", model_year)
        print("Price: ", price)
        print("\n")
        amount_of_cars += 1
    index += 1
    pagination_URL = "https://www.dba.dk/biler/biler/maerke-ford/side-" + str(index) 
    browser.get(pagination_URL)
    browser.implicitly_wait(1)
    # Browser will return to one of these URLs if we try to access an URL that doesn't
    # exist - for example if we try to reach page 100 and that page doesn't exist
    if browser.current_url == "https://www.dba.dk/biler/biler/maerke-ford/" or browser.current_url == "https://www.dba.dk/biler/biler/":
        break
    # Remove following line if you actually want the script to run through every
    # single page
    if (True):
        break

print("Amount of cars: ", amount_of_cars)
print("Amount of pages: ", index)

Model year:  2007
Price:  3.000 kr.


Model year:  2010
Price:  36.000 kr.


Model year:  2008
Price:  44.999 kr.


Model year:  2006
Price:  15.000 kr.


Model year:  2007
Price:  369.000 kr.


Model year:  2006
Price:  49.500 kr.


Model year:  2011
Price:  54.900 kr.


Model year:  1984
Price:  29.800 kr.


Model year:  2008
Price:  33.000 kr.


Model year:  2008
Price:  15.300 kr.


Model year:  2009
Price:  39.900 kr.


Model year:  1999
Price:  9.900 kr.


Model year:  2010
Price:  27.500 kr.


Model year:  2013
Price:  189.800 kr.


Model year:  2003
Price:  13.500 kr.


Model year:  2012
Price:  9.900 kr.


Model year:  2015
Price:  1 kr.


Model year:  2019
Price:  199.000 kr.


Model year:  1981
Price:  169.800 kr.


Model year:  2008
Price:  34.900 kr.


Model year:  2009
Price:  32.999 kr.


Model year:  2006
Price:  39.900 kr.


Model year:  2006
Price:  38.000 kr.


Model year:  2011
Price:  105.000 kr.


Model year:  2017
Price:  219.500 kr.


Model year:  2003
Price:  1

3. Åben de 5 dyreste biler med selenium i decending order og vis dem med et bar chart

In [74]:
import matplotlib.pyplot as plt
browser.get("https://www.dba.dk/biler/biler/")
browser.implicitly_wait(1)

sorting_panel = browser.find_element_by_class_name('sorting')
price_button_area = sorting_panel.find_elements_by_xpath('.//th[@colspan="1"]')[-1]
price_button = price_button_area.find_element_by_class_name('human-ref')
price_button.click()
browser.implicitly_wait(1)
sorting_panel = browser.find_element_by_class_name('sorting')
price_button_area = sorting_panel.find_elements_by_xpath('.//th[@colspan="1"]')[-1]
price_button = price_button_area.find_element_by_class_name('descendingCurrentOrder')
price_button.click()
# Or I could just use the link directly... https://www.dba.dk/biler/biler/?sort=price-desc

bulletin = browser.find_elements_by_class_name('dbaListing')
expensive_cars = {}
for index, element in enumerate(bulletin):
    description = element.find_elements_by_class_name('listingLink')[1].text
    stop_at_comma = re.compile(r'^[^,]*')
    mo = stop_at_comma.search(description)
    car_name = mo.group()
    price = element.find_element_by_xpath('.//td[@title="Pris"]').text
    expensive_cars[car_name] = price[:-4]
    amount_of_cars += 1
    if index >= 5:
        break
print(expensive_cars)
plt.bar(expensive_cars.keys(), expensive_cars.values(), width=0.9, align='center')

plt.axis([-1, len(expensive_cars.values()), 0, 100])
plt.title("5 most expensive cars", fontsize=12)
plt.ylabel("kr.", fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.xticks(rotation=10)
plt.show()


{'Porsche 918 Spyder 4': '10.125.000', 'Audi R8 5': '3.499.000', 'Bentley Continental GT 4': '2.399.500', 'Ferrari 458 4': '1.949.900', 'Ferrari F512 M 4': '1.953.600'}


<Figure size 640x480 with 1 Axes>